https://emdgroup.github.io/baybe/_modules/baybe/simulation/core.html#simulate_experiment

In [1]:
from baybe import Campaign
from baybe.objectives import SingleTargetObjective
from baybe.parameters import NumericalContinuousParameter
from baybe.searchspace import SearchSpace
from baybe.targets import NumericalTarget
from baybe.utils.botorch_wrapper import botorch_function_wrapper
from baybe.simulation import simulate_experiment
import numpy as np
import pandas as pd
import torch

In [2]:
# Parameters for a full simulation loop
N_MC_ITERATIONS = 10             # # of repeats 
N_DOE_ITERATIONS = 24            # # of "budget"/rounds in each repeat. Use 24 bc passing 1 initial data
BOUNDS = torch.tensor([[-5., 0.], [10., 15.]])
DIMENSION = 2

# definr branin function
def branin(x1, x2):
    y = float(
        (x2 - 5.1 / (4 * np.pi**2) * x1**2 + 5.0 / np.pi * x1 - 6.0) ** 2
        + 10 * (1 - 1.0 / (8 * np.pi)) * np.cos(x1)
        + 10
    )
    return y

WRAPPED_FUNCTION = branin

In [3]:
# define search space 
parameters = [
    NumericalContinuousParameter(
        name=f"x_{k+1}",
        bounds=(BOUNDS[0, k], BOUNDS[1, k]),
    )
    for k in range(DIMENSION)
]

# create search space
searchspace = SearchSpace.from_product(parameters=parameters)
objective = SingleTargetObjective(target=NumericalTarget(name="Target", mode="MIN"))

# construct campaign
campaign = Campaign(searchspace=searchspace, objective=objective)

In [4]:
# 10 random seeds and 10 initial points
random_seed_list = [17, 23, 42, 87, 99, 131, 518, 1047, 1598, 2024]
init_pt_list = [[7, 6], 
                [2, 0], 
                [9, 14], 
                [2, 8], 
                [10, 0], 
                [10, 15], 
                [5, 2], 
                [5, 13], 
                [7, 13], 
                [7, 2]]

In [5]:
# did not use the simulate_experiment function for better control

# for i in range(len(random_seed_list)):
#     random_seed = random_seed_list[i]
#     initial_data = pd.DataFrame(init_pt_list[i:1+i], columns=['x_1', 'x_2'])
#     initial_data['Target'] = WRAPPED_FUNCTION(initial_data['x_1'], initial_data['x_2'])

#     results = simulate_experiment(
#         campaign,
#         WRAPPED_FUNCTION,
#         batch_size=1,
#         n_doe_iterations=N_DOE_ITERATIONS,
#         initial_data=initial_data,
#         random_seed=random_seed,
#     )

In [6]:
from baybe.utils.random import set_random_seed
from copy import deepcopy

BATCH_SIZE = 1
results = pd.DataFrame()
for i in range(len(random_seed_list)):
    # set random seed
    set_random_seed(random_seed_list[i])

    # set initial point
    init_df = pd.DataFrame(init_pt_list[i:i+1], columns=['x_1', 'x_2'])
    init_df['Target'] = WRAPPED_FUNCTION(init_df['x_1'], init_df['x_2'])
    init_df["Random_Seed"] = random_seed_list[i]
    
    # copy campaign to avoid overwriting
    campaign_i = deepcopy(campaign)
    
    # add initial data to campaign
    campaign_i.add_measurements(init_df)

    for j in range(N_DOE_ITERATIONS): 
        recommendation = campaign_i.recommend(BATCH_SIZE)

        target_values = []
        for index, row in recommendation.iterrows():
            target_values.append(WRAPPED_FUNCTION(*row.to_list()))

        recommendation["Target"] = target_values
        recommendation["Random_Seed"] = random_seed_list[i]

        campaign_i.add_measurements(recommendation)

    # append campaign_i to results
    results = pd.concat([results, campaign_i.measurements])

C:\Users\MF\AppData\Local\Temp\ipykernel_21996\580042843.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y = float(
c:\Users\MF\anaconda3\envs\BayBE\lib\site-packages\botorch\models\transforms\outcome.py:289: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ..\aten\src\ATen\native\ReduceOps.cpp:1807.)
  stdvs = Y.std(dim=-2, keepdim=True)
c:\Users\MF\anaconda3\envs\BayBE\lib\site-packages\botorch\models\utils\assorted.py:194: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ..\aten\src\ATen\native\ReduceOps.cpp:1807.)
  Ymean, Ystd = torch.mean(Y, dim=-2), torch.std(Y, dim=-2)
C:\Users\MF\AppData\Local\Temp\ipykernel_21996\580042843.py:9: Fu

In [7]:
# campaign_i.measurements

In [8]:
results

,x_1,x_2,Target,Random_Seed,BatchNr,FitNr
0,7.000000,6.000000,40.382900,17,1,1.0
1,-4.165027,14.664207,5.045011,17,2,2.0
2,-5.000000,10.723315,54.507634,17,3,3.0
3,-1.470913,15.000000,51.655116,17,4,4.0
4,-1.040772,0.000000,75.637823,17,5,5.0
...,...,...,...,...,...,...
20,3.095091,2.119372,0.445198,2024,21,21.0
21,0.963598,5.075685,15.718127,2024,22,22.0
22,-2.923190,11.982049,0.676956,2024,23,23.0
23,10.000000,1.902905,3.153254,2024,24,24.0


In [10]:
dff = results['Target'].to_numpy().reshape(N_MC_ITERATIONS, N_DOE_ITERATIONS+1)
np.savetxt('fix-BayBE.csv', dff)